# Acelerando Python con Cython

La implementación más utilizada del intérprete de Python está escrita en C y se llama [CPython](https://github.com/python/cpython). 

:::{note}

También existen otras implementaciones de Python, como [PyPy](https://www.pypy.org/) (incorpora compilación JIT), [Jython](https://www.jython.org/) (escrito en Java) o IronPython. 

:::

CPython compila el código escrito en Python en un código de máquina (binario) de forma transparente al usuario. Luego CPython interpreta el binario. 

:::{note}

Si es de vuestro interés, es posible estudiar el código binario usando el módulo [dis](https://docs.python.org/3/library/dis.html) de Python.

:::

Por lo tanto, si sabemos C, podemos usar la [API de Python/C](https://docs.python.org/3/c-api/index.html) para:

1. Escribir módulos de C que puedan llamarse desde Python
1. Hacer interfaces entre código C y Python

Sin embargo, la API es un poco complicada, por lo que han surgido alternativas menos tediosas para lograr estos objetivos, como por ejemplo [ctypes](https://docs.python.org/3/library/ctypes.html), [CFFI](https://cffi.readthedocs.io/en/latest/), [SWIG](http://swig.org/) y [Cython](https://cython.org/).

En este capítulo veremos en detalle esta última.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## Cython: C Extensions for Python

Recordemos que **Python** es un lenguaje interpretado con tipos dinámicos. Esto hace que cada operación tenga un overhead. Por ejemplo:

```python
    z = x + y
    # overhead: Inferir el tipo de x
    # overhead: Inferir el tipo de y
    # Hacer la operación suma
    # overhead: Darle el tipo adecuado a z
```

en cambio, **Cython** es un lenguaje de programación que le agrega a Python algunas propiedades de C y C++, una de ellas son los **tipos estáticos**, como muestra el siguiente ejemplo:

```python
    int x = 1
    int y = 2
    int z = x + y
    # No hay que inferir el tipo de x, y, z
```

Esto hace que Cython sea menos flexible pero decenas de veces más rápido que Python. En términos de sintaxis Cython es casi tan simple como Python. Sin embargo a diferencia de Python, el lenguaje Cython debe compilarse.

- El compilador de Cython convierte el código fuente en código C 
- Luego el código C se compila como un módulo de Python con la implementación **CPython**

:::{important}

Una vez compilado el código escrito en Cython este puede llamarse desde Python.

:::

¿Por qué considerar Cython?

- Cython es casi tan simple como Python y casi tan rápido como C
- Con Cython se pueden llamar funciones y librerías de C
- Cython se integra de buena manera con NumPy

Por ende Cython es muy atractivo para proyectos que usen Python y tengan requisitos de alto rendimiento. Estudiaremos la sintaxis de Cython mediante algunos ejemplos.

**Ejemplo** A lo largo de esta lección utilizaremos como ejemplo el cálculo de la **Distancia euclidiana todos-con-todos**

Sea un conjunto de con $N$ datos con $D$ atributos (matriz de NxD) donde queremos calcular la distancia euclidiana de cada dato con todos los demás, es decir una matriz donde el elemento $ij$ es: 

$$
\text{dist}_{ij} = \sqrt {\sum_{k=1}^D (x_{ik} - x_{jk})^2}
$$

A continuación se muestran dos códigos que cumplen este propósito y obtienen un resultado equivalente:

- El primero usa "Python puro" y calcula las distancias en un triple bucle
- El segundo usa operaciones vectoriales de NumPy 

Estudie ambos códigos y asegúrese de comprenderlos antes de continuar.

In [2]:
data = np.random.randn(1000, 2)

def distancia_pares(data):    
    N, D = data.shape
    dist = np.zeros(shape=(N, N))
    for i in range(N):
        for j in range(i+1, N):
            for k in range(D):
                dist[i, j] += (data[i, k] - data[j, k])**2
            dist[i, j] = np.sqrt(dist[i, j])
            dist[j, i] = dist[i, j]
    return dist
            
def distancia_pares_numpy(data):
    return np.sqrt(np.sum((data.reshape(-1, 1, 2) - data.reshape(1, -1, 2))**2, axis=-1))



El resultado de ambas rutinas es equivalente:

In [3]:
np.allclose(distancia_pares(data), distancia_pares_numpy(data))

True

El tiempo que toma cada uno en completar es:

In [4]:
time_pure = %timeit -r3 -n1 -q -o distancia_pares(data)
time_pure.average

0.5891087003207455

In [5]:
time_numpy = %timeit -r10 -n5 -q -o distancia_pares_numpy(data)
time_numpy.average

0.013841385659761726

Por lo tanto, NumPy es:

In [6]:
time_pure.average/time_numpy.average

42.56139629382213

veces más rápido que Python puro

## Cython desde IPython/Jupyter

Instale cython en su ambiente de conda con:

    conda install cython

Luego en IPython podemos cargar la extensión `cython` como se muestra a continuación.

In [7]:
%load_ext cython

con esto tendremos disponible la magia de bloque `%%cython`

:::{important}

Un bloque donde se use esta magia acepta lenguaje cython y se compila al ejecutarlo. Luego podremos llamar las funciones de ese bloque desde otros bloques regulares de Python.

:::

Si surgen errores de compulación estos aparecen en la salida del bloque. 

:::{note}

El bloque que empieza con `%%cython` está "desconectado" del resto del cuadernillo, por lo que debe tener sus propios `import`.

:::

La magia tiene las siguientes opciones:

- `-a` (annotate) retorna un profile linea a linea indicando con amarillo las llamadas a CPython (mientras más llamadas más lento es nuestro código)
- `-+` Usar C++ en lugar de C
- `-c` Argumentos de compilación
- `-l` librerías para linkear a nuestro código
- `-L` directorio con librerías
- `-I` directorio con cabeceras (include)   


Veamos que ocurre al agregarle la magia al ejemplo anterior.

In [8]:
%%cython

import numpy as np

def distancia_pares_cython_ingenuo(data):    
    N, D = data.shape
    dist = np.zeros(shape=(N, N))
    for i in range(N):
        for j in range(i+1, N):
            for k in range(D):
                dist[i, j] += (data[i, k] - data[j, k])**2
            dist[i, j] = np.sqrt(dist[i, j])
            dist[j, i] = dist[i, j]
    return dist

In [9]:
np.allclose(distancia_pares_numpy(data), distancia_pares_cython_ingenuo(data))

True

In [10]:
time_cython_naive = %timeit -r3 -n1 -q -o distancia_pares_cython_ingenuo(data)
time_cython_naive.average

0.5217682596606513

In [11]:
time_pure.average/time_cython_naive.average

1.1290619722707762

In [12]:
time_numpy.average/time_cython_naive.average

0.026527841438196937

El resultado es idéntico pero como no hemos hecho ningún cambio, el tiempo de ejecución es sólo levemente mejor que la versión en Python puro.

A continuación veremos como "cythonizar" nuestro código para ganar rendimiento.

## Mejora 1: Definiendo tipos en Cython

En Cython se definen tipos estáticos con el keyword `cdef` seguido del tipo y luego el nombre. Por ejemplo una variable de tipo `double` llamada `mi_variable` sería:

```cython
cdef double mi_variable 
```
    
Para los arreglos (ndarray) se usan ["memory-views"](https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html#memoryviews). Por ejemplo declarar un *memory-view* para una arreglo de tres dimensiones sería:

```cython
cdef double [:, :, :] mi_arreglo
```    

Y se puede ganar un poco más de rendimiento usando el operador `::1` para especificar si el arreglo es *row-major* (estilo C)

```cython
cdef double [:, :, ::1] mi_arreglo
```    

o *column-major* (estilo Fortran)

```cython
cdef double [::1, :, :] mi_arreglo
```

Veamos como queda el ejemplo anterior introduciendo tipos estáticos.

In [13]:
%%cython -a

import numpy as np

def distancia_pares_cython_estatico(double [:, ::1] data):
    # Definimos el tipo de N, D, dist y data
    cdef int N = data.shape[0]
    cdef int D = data.shape[1]
    dist = np.zeros(shape=(N, N), dtype=np.double)
    cdef double [:, ::1] dist_view = dist
    # También definimos los índices, se puede usar int o Py_ssize_t 
    cdef int i, j, k
    for i in range(N):
        for j in range(i+1, N):
            for k in range(D):
                dist_view[i, j] += (data[i, k] - data[j, k])**2
            dist_view[i, j] = np.sqrt(dist_view[i, j])
            dist_view[j, i] = dist_view[i, j]
    return dist

In [14]:
np.allclose(distancia_pares_numpy(data), distancia_pares_cython_estatico(data))

True

In [15]:
time_cython_static = %timeit -r10 -n5 -o -q distancia_pares_cython_estatico(data)
time_cython_static.average

0.14667874137870968

Speed-up con respecto a Python puro:

In [16]:
time_pure.average/time_cython_static.average

4.016319575579984

Speed-up con respecto a NumPy:

In [17]:
time_numpy.average/time_cython_static.average

0.09436531517559636

Con solo definir el tipo de `data`, `dist`, `N`, `D` y los índices hemos obtenido un *speed-up* importante con respecto a "Python puro", aunque sigue siendo más lento que NumPy.

Si observaramos el código anotado por cython (compilado con `-a`) y revisamos la cantidad de llamadas a CPython de cada línea:

<img src="img/cython1.png" width="700">

Podemos notar que la linea 16 es particularmente conflictiva: 

- involucra una gran cantidad de instrucciones
- se llama NxN veces

esto se debe a que estamos usando la función de NumPy `np.sqrt`. Podemos obtener un rendimiento mucho mayor si usamos la implementación de raíz cuadrada de C.

## Mejora 2: Llamando funciones de C desde Cython

Es posible llamar funciones de C desde Cython de forma sencilla. Consideremos como ejemplo la función `sqrt` de la [librería matemática estándar de C](http://pubs.opengroup.org/onlinepubs/9699919799/basedefs/math.h.html). 

Para usar `sqrt` desde cython necesitamos:

- Importar la función `sqrt` desde la cabecera `math.h`. Esto se hace con el keyword `cdef extern from`
- Compilar contra `libm`

Veamos como queda nuestro ejemplo con esta modificación.

In [18]:
%%cython -l m

import numpy as np

cdef extern from "math.h":
    double sqrt(double)

def distancia_pares_cython_sqrtC(double [:, ::1] data):
    # Definimos el tipo de N, D, dist y data
    cdef int N = data.shape[0]
    cdef int D = data.shape[1]
    dist = np.zeros(shape=(N, N), dtype=np.double)
    cdef double [:, ::1] dist_view = dist
    # También definimos los índices, se puede usar int o Py_ssize_t 
    cdef Py_ssize_t i, j, k
    for i in range(N):
        for j in range(i+1, N):
            for k in range(D):
                dist_view[i, j] += (data[i, k] - data[j, k])**2
            dist_view[i, j] = sqrt(dist_view[i, j])
            dist_view[j, i] = dist_view[i, j]
    return dist

In [19]:
np.allclose(distancia_pares_numpy(data), distancia_pares_cython_sqrtC(data))

True

In [20]:
time_static_cfun = %timeit -r10 -n10 -q -o distancia_pares_cython_sqrtC(data)
time_static_cfun.average

0.0017283304408192636

In [21]:
time_numpy.average/time_static_cfun.average

8.008529695976788

Con esta simple modificación hemos obtenido un tiempo de ejecución incluso menor que la implementación en NumPy.

:::{important}

Utilizando `extern` podemos hacer interfaces [entre Python y casi cualquier código escrito en C](https://cython.readthedocs.io/en/latest/src/tutorial/clibraries.html).

:::

## Mejora 3: Deshabilitando comprobaciones para ir aun más rápido

Podemos hacer nuestro código más rápido (y más inseguro) deshabilitando las verificaciones que Python realiza por defecto.

En Cython esto se logra usando decoradores que funcionan como directivas de compilación. Las opciones disponibles se encuentrán [aquí](https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html#compiler-directives). En este caso particular deshabilitaremos tres verificaciones: *boundscheck* y *wraparound*.

Al deshabilitarlas el código no comprobará si escribimos fuera del arreglo y tampoco traducirá índices negativos.

In [22]:
%%cython -l m
import cython
cimport numpy as npc
import numpy as np

# Por conveniencia podemos definir el tipo de data y dist con 
cdef extern from "math.h":
    double sqrt(double)

# Deshabilitamos las comprobaciones de Python:
@cython.boundscheck(False)
@cython.wraparound(False)
def distancia_pares_cython(double [:, ::1] data):
    cdef int N = data.shape[0]
    cdef int M = data.shape[1]
    dist = np.zeros(shape=(N, N), dtype=np.double)
    cdef double [:, ::1] dist_view = dist
    cdef Py_ssize_t i, j, k
    for i in range(N):
        for j in range(i+1, N):
            for k in range(M):
                dist_view[i, j] += (data[i, k] - data[j, k])**2
            dist_view[i, j] = sqrt(dist_view[i, j])
            dist_view[j, i] = dist_view[i, j]
    return dist

In [23]:
np.allclose(distancia_pares_numpy(data), distancia_pares_cython(data))

True

In [24]:
time_cython_static_cfun_nochecks = %timeit -r30 -n30 -q -o distancia_pares_cython(data)
time_cython_static_cfun_nochecks.average

0.0012998963077552616

In [25]:
time_numpy.average/time_cython_static_cfun_nochecks.average

10.648069063034615

Una modificación marginal en el código que aumenta el speed-up con respecto al caso anterior.

##  Mejora 4: Mayor flexibilidad con tipos de Cython fusionados 

Lo que hemos perdido al usar Cython con respecto a Python es la **flexibilidad**.

Si utilizamos un argumento que no sea `double` nuestra función en Cython retornará un error. Es nuestra responsabilidad evitar que esto ocurra.

In [26]:
data_float32 = data.astype(np.float32)

Si ejecutaramos la función `distancia_pares_cython` con datos de tipo float32 resultaría en una excepción.

<img src="img/cython2.png" width="600">


Podemos resolver este tipo de problemas con los "tipos fusionados" de Cython. Si queremos fusionar dos tipos de datos "tipo1" y "tipo2" bajo el nombre "tipo3", lo declaramos con:

```cython
ctypedef fused tipo3:
    tipo1
    tipo2
```

Con esto Cython creará, de forma transparente, dos funciones en lugar de una. Cython ofrece [algunos tipos fusionados listos para utilizar](https://cython.readthedocs.io/en/latest/src/userguide/fusedtypes.html#built-in-fused-types). Este es el caso del tipo fusionado `floating`, que combina números flotantes con precisión doble (float64) y simple (float32).

Modifiquemos la función `distancia_pares_cython` para que acepte los tipos double e float como un tipo fusionado. En este caso no debemos olvidar importar la definición de `sqrtf` de `math.h`.

In [27]:
%%cython -l m
import cython
from cython cimport floating
cimport numpy as npc
import numpy as np

    
cdef extern from "math.h":
    float sqrtf(float) #Definición para float32
    double sqrt(double) # Definición para float64


@cython.boundscheck(False)
@cython.wraparound(False)
def distancia_pares_cython_multitipo(floating [:, ::1] data):
    cdef int N = data.shape[0]
    cdef int M = data.shape[1]
    # Comprobamos el tipo antes de crear el arreglo de numpy
    if floating is double:
        dist = np.zeros(shape=(N, N), dtype=np.float64)        
    else:
        dist = np.zeros(shape=(N, N), dtype=np.float32)
    
    cdef floating [:, ::1] dist_view = dist
    cdef Py_ssize_t i, j, k
    for i in range(N):
        for j in range(i+1, N):
            for k in range(M):
                dist_view[i, j] += (data[i, k] - data[j, k])**2
            if floating is double:
                dist_view[i, j] = sqrt(dist_view[i, j])
            else:
                dist_view[i, j] = sqrtf(dist_view[i, j])
            dist_view[j, i] = dist_view[i, j]
    return dist

Los resultados son equivalentes a NumPy tanto si se usa precisión doble o simple:

In [28]:
np.allclose(distancia_pares_numpy(data),
            distancia_pares_cython_multitipo(data))

True

In [29]:
np.allclose(distancia_pares_numpy(data_float32),
            distancia_pares_cython_multitipo(data_float32))

True

In [30]:
time_numpy_f32 = %timeit -r10 -n10 -q -o distancia_pares_numpy(data_float32)
time_numpy_f32.average

0.01272030138177797

In [31]:
time_cython_f32 = %timeit -r30 -n30 -q -o distancia_pares_cython_multitipo(data_float32)
time_cython_f32.average

0.0005650823929398838

In [32]:
time_numpy_f32.average/time_cython_f32.average

22.51052508573067

El speed-up con double es pracitcamente equivalente al caso anterior.

In [33]:
time_cython = %timeit -r30 -n30 -q -o distancia_pares_cython_multitipo(data)
time_numpy.average/time_cython.average

10.535512549007606

## Usando Cython fuera de jupyter

:::{seealso}

Lecturas sobre temas específicos de Cython:

- [Incluyendo código Cython en un módulo de Python](https://cython.readthedocs.io/en/latest/src/quickstart/build.html#building-a-cython-module-using-distutils)
- [Haciendo profiling de código Cython](https://cython.readthedocs.io/en/latest/src/tutorial/profiling_tutorial.html)
- [Malloc/Free en Cython](https://cython.readthedocs.io/en/latest/src/tutorial/memory_allocation.html)
- Libro [Cython: A Guide for Python Programmers](https://pythonbooks.org/cython-a-guide-for-python-programmers/)

:::